In [1]:
import numpy as np

## First order high-pass filter

### Continuous transfer function
The high-pass filter transfer function is
$$H(s) = \frac{s}{s + \omega_0}$$

### Discrete transfer function
Computing the discrete transfer function using Tustin's method (bilinear transform), set $s = \frac{2}{\Delta t} \left( \frac{1-z^{-1}}{1+z^{-1}} \right)$, so
\begin{align*} 
H(z) &= \frac{ \frac{2}{\Delta t} \frac{1-z^{-1}}{1+z^{-1}} }{\frac{2}{\Delta t} \frac{1-z^{-1}}{1+z^{-1}}  + \omega_0} \\
     &= \frac{1 + (-1)z^{-1}}{(1 + \frac{\Delta t \omega_0}{2}) + ( \frac{\Delta t \omega_0}{2} - 1) z^{-1} }
 \end{align*}


We want to find the filter coefficients for the discrete update:
$$y[n] = a_1 y[n-1] + a_2 y[n-2] + ... + b_0 x[n] + b_1 x[n-1] + ...$$

The coefficients can be taken directly from the discrete transfer function of the filter in the form:
$$H(z) = \frac{b_0 + b_1 z^{-1} + b_2 z^{-2} + \ldots}{1 - a_1 z^{-1} - a_2 z^{-2} + \ldots}$$

Setting $\alpha = \omega_0 \Delta t$, the transfer function is
\begin{align*} 
H(z) &= \frac{1 + (-1)z^{-1}}{(1 + \alpha/2) + ( \alpha/2 - 1) z^{-1} } \\
  &= \frac{1/(1 + \alpha/2) + [-1/(1 + \alpha/2)]z^{-1}}{ 1 + ( \alpha/2 - 1)/(1 + \alpha/2) z^{-1} }
 \end{align*}


$$ b_0 = \frac{1}{1 + \alpha /2}\quad  b_1 = - \frac{1}{1 + \alpha /2} \quad \text{and} \quad a_1 = - \frac{ \alpha/2 - 1}{1 + \alpha/2} $$

### Arduino code
On the arduino code, we compute the coefficients as

<pre>
float alpha = omega0*dt;
float alphaFactor = 1/(1 + alpha/2.0);
a[0] = -(alpha/2.0 - 1)*alphaFactor;
b[0] = alphaFactor;
b[1] = -alphaFactor;        
</pre>

In [2]:
# Example coefficients for testing
dt = 1/1.0e3;
omega0 = 30*2*np.pi;
alpha = omega0*dt;
a = [0];
b = [0,0];
alpha = omega0*dt;
alphaFactor = 1/(1 + alpha/2.0);
a[0] = -(alpha/2.0 - 1)*alphaFactor;
b[0] = alphaFactor;
b[1] = -alphaFactor;  

print(b)
print(a)

[0.9138698004564532, -0.9138698004564532]
[0.8277396009129062]


<hr>

## Second order high-pass Butterworth filter

### Continuous transfer function

The $n^{\text{th}}$ Butterworth low-pass filter transfer function with $\omega_c = 1$ can be written as (see https://en.wikipedia.org/wiki/Butterworth_filter)
$$H(s) = \frac{1}{\sum_0^{n} \beta_k s^k}$$
where $n$ is the order of the filter. The coefficients are given by the recursion formula:
$$\beta_{k+1} = \frac{\cos( k \gamma )}{\sin((k+1)\gamma)} \beta_k$$
with $\beta_0 = 1$ and $\gamma = \frac{\pi}{2n}$.

The $n^{\text{th}}$ order high-pass Butterworth filter can be written as 
$$H(s) = \frac{s^n}{\sum_0^{n} c_{j} s^{j}}$$
with 
 $c_j = \beta_{n-j} {\omega_c}^{n-j}$
 
This implies that a second order filter is:
$$H(s) = \frac{s^2}{ c_0 + c_1 s + c_2 s^2}$$ 
with  $c_j = \beta_{2-j} {\omega_c}^{2-j}$. Using the recursion formula with $n = 2$, the Butterworth coefficients are:
$$\beta_0 = 1 \qquad \beta_1 = \sqrt{2} \qquad \beta_2 =  1$$

### Discrete transfer function

Computing the discrete transfer function using Tustin's method (bilinear transform), set $s = \frac{2}{\Delta t} \left( \frac{1-z^{-1}}{1+z^{-1}} \right)$, so

\begin{align*} 
H(z) &=\frac{\left[ \frac{2}{\Delta t} \left( \frac{1-z^{-1}}{1+z^{-1}} \right) \right]^2}{ c_0 + c_1 \left[ \frac{2}{\Delta t} \left( \frac{1-z^{-1}}{1+z^{-1}} \right) \right] + c_2 \left[ \frac{2}{\Delta t} \left( \frac{1-z^{-1}}{1+z^{-1}} \right) \right]^2} \\
     &= \frac{ 4 ( 1 - z^{-1})^2 }{(1 - z^{-1})^2 \Delta t^2 c_0 + 2 c_1 \Delta t (1-z^{-1})(1+z^{-1}) + 4 c_2 (1-z^{-1})^2 } \\
    &= \frac{4 - 8 z^{-1} + 4 z^{-2}}{ (c_0 \Delta t^2 + 2 c_1 \Delta t + 4 c_2) + (2 c_0 \Delta t^2 - 8 c_2) z^{-1} + (c_0 \Delta t^2 - 2 c_1 \Delta t  + 4 c_2 )z^{-2} }
 \end{align*}
Setting $D = c_0 \Delta t^2 + 2 c_1 \Delta t + 4 c_2$, this is

$$ H(z) = \frac{4/D + (-8/D) z^{-1} + (4/D) z^{-2}}{ 1 + [(2 c_0 \Delta t^2 - 8 c_2)/D] z^{-1} + [(c_0 \Delta t^2 - 2 c_1 \Delta t  + 4 c_2 )/D] z^{-2} } $$


We want to find the filter coefficients for the discrete update:
$$y[n] = a_1 y[n-1] +a_2 y[n-2] + ... + b_0 x[n] + b_1 x[n-1] + ...$$

The coefficients can be taken directly from the discrete transfer function of the filter in the form:
$$H(z) = \frac{b_0 + b_1 z^{-1} + b_2 z^{-2} + \ldots}{1 - a_1 z^{-1} - a_2 z^{-2} + \ldots}$$

So the discrete update coefficients are

$$ b_0 = 4/D \quad b_1 = -8/D \quad b_2 = 4/D $$

$$ a_1 = -(2 c_0 \Delta t^2 - 8 c_2)/D \quad a_2 = -(c_0 \Delta t^2 - 2 c_1 \Delta t  + 4 c_2 )/D $$

Where $\{c_i\} = \{ \omega_c^2 , \sqrt{2} \omega_c, 1 \}$

### Arduino code
On the arduino code, we compute the coefficients as

<pre>
float alpha = omega0*dt;
float dtSq = dt*dt;
float c[] = {omega0*omega0, sqrt(2)*omega0, 1};
float D = c[0]*dtSq + 2*c[1]*dt + 4*c[2];

b[0] = 4.0/D;
b[1] = -8.0/D;
b[2] = 4.0/D;

a[0] = -(2*c[0]*dtSq - 8*c[2])/D;
a[1] = -(c[0]*dtSq - 2*c[1]*dt + 4*c[2])/D;
</pre>

In [3]:
# Example coefficients for testing
dt = 1/1.0e3;
omega0 = 20*2*np.pi;
alpha = omega0*dt;
dtSq = dt*dt;
c= [omega0*omega0, np.sqrt(2)*omega0, 1];
D = c[0]*dtSq + 2*c[1]*dt + 4*c[2];

b = [0,0,0]
b[0] = 4.0/D;
b[1] = -8.0/D;
b[2] = 4.0/D;

a = [0,0]
a[0] = -(2*c[0]*dtSq - 8*c[2])/D;
a[1] = -(c[0]*dtSq - 2*c[1]*dt + 4*c[2])/D;

print(b)
print(a)

[0.9150759211230759, -1.8301518422461518, 0.9150759211230759]
[1.822926692375394, -0.8373769921169094]
